# Using EcoFOCIpy to process raw field data

## AQ2301 (Fall Mooring Cruise - Aquila)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [1]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [2]:
sample_data_dir = '/Users/bell/ecoraid/2023/CTDcasts/aq2301/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [3]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'AQ2301' #no hyphens
cruise_meta_file = sample_data_dir+'logs/AQ2301.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTD.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd001.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd002.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd003.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd004.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd005.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd006.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd007.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd008.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd009.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd010.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd011.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd012.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd013.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd014.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd015.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd016.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd017.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd018.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd019.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd020.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd021.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd022.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd023.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd024.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd025.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd026.btl
Processing /

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd028.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd029.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd030.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd031.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd032.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd033.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd034.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd035.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd036.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd037.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd038.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd039.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd040.btl
Processing /

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd042.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/aq2301/rawconverted/ctd043.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])


In [4]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['ctd001.btl'].describe()

,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sal00,sal11,sigma-t00,sigma-t11,sbeox1ml/l,sbeox1ps,sbox1mm/kg,t090c,t190c,prdm,fleco-afl,turbwetbb0,scan,datetime
count,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12
mean,5.960400,90.775667,259.806250,31.808483,31.813642,24.601275,24.606025,6.189808,94.270000,269.806083,9.080575,9.075842,76.200500,0.552025,0.602766,20395.083333,2023-08-15 14:59:48.166667008
min,5.407100,79.759000,235.577000,31.511200,31.514500,24.170400,24.173100,5.620900,82.909000,244.893000,7.642700,7.642900,2.145000,0.188100,0.121350,12580.000000,2023-08-15 14:54:22
25%,5.425825,80.041250,236.394500,31.537725,31.542025,24.216250,24.219625,5.624350,82.970500,245.043750,7.649175,7.649925,25.904000,0.218375,0.313660,12815.250000,2023-08-15 14:54:32.249999872
50%,6.205700,95.410000,270.532500,31.777800,31.783550,24.514950,24.520050,6.462100,99.351000,281.709000,9.516350,9.512350,62.169000,0.423450,0.389795,20299.000000,2023-08-15 14:59:44.500000
75%,6.317725,98.158750,275.436000,32.112775,32.116325,25.056775,25.059375,6.559100,101.997500,285.958500,10.202750,10.202775,147.673500,0.940825,1.079300,25642.750000,2023-08-15 15:03:26.750000128
max,6.392100,99.759000,278.735000,32.114500,32.118000,25.059000,25.061700,6.620500,103.323000,288.695000,10.354300,10.353300,148.660000,1.082700,1.130000,31805.000000,2023-08-15 15:07:44
std,0.429092,8.673226,18.796295,0.266835,0.267275,0.389377,0.389933,0.447984,9.040037,19.622859,1.185013,1.186469,59.321311,0.364253,0.384290,6970.707365,NaN


## Create BTL report file

In [5]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd001':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [6]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'AQ2301',
 'CruiseID_Historic': '',
 'CruiseID_Alternates': '',
 'Project_Leg': '',
 'Vessel': 'F/V Aquila',
 'ShipID': 'AQ',
 'StartDate': datetime.date(2023, 8, 15),
 'EndDate': datetime.date(2023, 9, 10),
 'Project': 'EcoFOCI, NMML',
 'ChiefScientist': 'Ryan McCabe',
 'StartPort': 'Kodiak, AK',
 'EndPort': 'Dutch Harbor, AK',
 'CruiseLocation': 'Bering Sea',
 'Description': 'Fall Mooring Cruise',
 'CruiseYear': 2023,
 'ctdlogs_pdf_name': 'AQ2301_CTD_Logs.pdf'}

In [7]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD001']

{'id': 54238,
 'Vessel': 'F/V Aquila',
 'CruiseID': 'AQ2301',
 'Project_Leg': '',
 'UniqueCruiseID': 'AQ2301',
 'Project': 'EcoFOCI Fall Mooring Cruise',
 'StationNo_altname': 's1h1',
 'ConsecutiveCastNo': 'CTD001',
 'LatitudeDeg': 57,
 'LatitudeMin': 43.53,
 'LongitudeDeg': 152,
 'LongitudeMin': 17.373,
 'GMTDay': 15,
 'GMTMonth': 'Aug',
 'GMTYear': 2023,
 'GMTTime': 78900,
 'DryBulb': -99.9,
 'RelativeHumidity': -999,
 'WetBulb': -99.9,
 'Pressure': -9999,
 'SeaState': '',
 'Visibility': '',
 'WindDir': -999,
 'WindSpd': -99.0,
 'CloudAmt': '',
 'CloudType': '',
 'Weather': '',
 'SurfaceTemp': -99.9,
 'BottomDepth': 156,
 'StationNameID': '22CB01',
 'MaxDepth': 148,
 'InstrumentSerialNos': '',
 'Notes': "Bottom Temp ~7.5C, must have drifted, bottom depth has changed alot\r\nbottle 12 misfired, no samples\r\nbottle 1 oxygen mistake - don't run",
 'NutrientBtlNiskinNo': '1;2;3;4;5;6;7;8;9;10;11;12',
 'NutrientBtlNumbers': '1;;;;2;3;4;5;6;7;8;',
 'OxygenBtlNiskinNo': '',
 'OxygenBtlNumb

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [8]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [9]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['ctd001.btl'] = cruise_data['ctd001.btl'].rename(columns={
                        't090c':'temperature_ch1',
                        't190c':'temperature_ch2',
                        'sal00':'salinity_ch1',
                        'sal11':'salinity_ch2',
                        'sbox0mm/kg':'oxy_conc_ch1',
                        'sbeox0ml/l':'oxy_concM_ch1',
                        'sbox1mm/kg':'oxy_conc_ch2',
                        'sbeox1ml/l':'oxy_concM_ch2',
                        'sbeox0ps':'oxy_percentsat_ch1',
                        'sbeox1ps':'oxy_percentsat_ch2',
                        'sigma-t00':'sigma_t_ch1',
                        'sigma-t11':'sigma_t_ch2',
                        'cstarat0':'Attenuation',
                        'cstartr0':'Transmittance',
                        'fleco-afl':'chlor_fluorescence',
                        'turbwetntu0':'turbidity',
                        'empty':'empty', #this will be ignored
                        'prdm':'Pressure [dbar]',
                        'flag':'flag'})

cruise_data['ctd001.btl'].sample()

,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,salinity_ch1,salinity_ch2,sigma_t_ch1,sigma_t_ch2,oxy_concM_ch2,oxy_percentsat_ch2,oxy_conc_ch2,temperature_ch1,temperature_ch2,Pressure [dbar],chlor_fluorescence,turbwetbb0,scan,datetime,cast
bottle,,,,,,,,,,,,,,,,,,
2.0,5.4202,79.949,236.147,32.1145,32.118,25.059,25.0617,5.6209,82.909,244.893,7.6427,7.6429,148.66,0.1881,1.0835,12673.0,2023-08-15 14:54:26,ctd001


In [10]:
cruise_data['ctd001.btl'].columns

Index(['oxy_concM_ch1', 'oxy_percentsat_ch1', 'oxy_conc_ch1', 'salinity_ch1',
       'salinity_ch2', 'sigma_t_ch1', 'sigma_t_ch2', 'oxy_concM_ch2',
       'oxy_percentsat_ch2', 'oxy_conc_ch2', 'temperature_ch1',
       'temperature_ch2', 'Pressure [dbar]', 'chlor_fluorescence',
       'turbwetbb0', 'scan', 'datetime', 'cast'],
      dtype='object')

## Add institutional meta-information


In [11]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [12]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['ctd001.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [13]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 12)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 ... 9.0 10.0 11.0 12.0
Data variables: (12/18)
    oxy_concM_ch1       (bottle) float64 5.428 5.42 5.407 ... 6.321 6.32 6.392
    oxy_percentsat_ch1  (bottle) float64 80.07 79.95 79.76 ... 98.39 98.43 99.76
    oxy_conc_ch1        (bottle) float64 236.5 236.1 235.6 ... 275.6 275.6 278.7
    salinity_ch1        (bottle) float64 32.11 32.11 32.11 ... 31.53 31.53 31.51
    salinity_ch2        (bottle) float64 32.12 32.12 32.12 ... 31.54 31.53 31.51
    sigma_t_ch1         (bottle) float64 25.06 25.06 25.06 ... 24.21 24.2 24.17
    ...                  ...
    Pressure [dbar]     (bottle) float64 147.8 148.7 148.6 ... 19.55 10.27 2.145
    chlor_fluorescence  (bottle) float64 0.2195 0.1881 0.1983 ... 1.022 0.9344
    turbwetbb0          (bottle) float64 1.078 1.083 1.13 ... 0.3033 0.1213
    scan                (bottle) float64 1.258e+04 1.267e+04 ... 3.18e+04
    datetime            (bottle) datetime64[ns] 2023-08-15T14:54:22 ... 2023-...
    cast                (bottle) object 'ctd001' 'ctd001' ... 'ctd001' 'ctd001'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [14]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [15]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['ctd001.btl'].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [16]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 12, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 ... 9.0 10.0 11.0 12.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/18)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 5.428 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 80.07 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 236.5 ......
    salinity_ch1        (latitude, longitude, time, bottle) float64 32.11 ......
    salinity_ch2        (latitude, longitude, time, bottle) float64 32.12 ......
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 25.06 ......
    ...                  ...
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 147.8 ......
    chlor_fluorescence  (latitude, longitude, time, bottle) float64 0.2195 .....
    turbwetbb0          (latitude, longitude, time, bottle) float64 1.078 ......
    scan                (latitude, longitude, time, bottle) float64 1.258e+04...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd001' ....

In [17]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD001')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 12, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 ... 9.0 10.0 11.0 12.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/18)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 5.428 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 80.07 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 236.5 ......
    salinity_ch1        (latitude, longitude, time, bottle) float64 32.11 ......
    salinity_ch2        (latitude, longitude, time, bottle) float64 32.12 ......
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 25.06 ......
    ...                  ...
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 147.8 ......
    chlor_fluorescence  (latitude, longitude, time, bottle) float64 0.2195 .....
    turbwetbb0          (latitude, longitude, time, bottle) float64 1.078 ......
    scan                (latitude, longitude, time, bottle) float64 1.258e+04...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd001' ....
Attributes: (12/44)
    CruiseID:                      AQ2301
    CruiseID_Historic:             
    CruiseID_Alternates:           
    Project_Leg:                   
    Vessel:                        F/V Aquila
    ShipID:                        AQ
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2023-12-14T07:36:58Z
    date_modified:                 
    QC_indicator:                  unqcd

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [18]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncC

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history